In [1]:
!pip install spacy transformers torch
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 116.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#!gcloud auth application-default login

In [ ]:
#Embedding generated based consudering the sentence where entity occurs

In [3]:
import spacy
import google.generativeai as genai
from typing import List, Dict

# --- Configure the Gemini SDK (assumes API key or ADC is set up) ---
#genai.configure()

# --- Load spaCy NER model ---
nlp = spacy.load("en_core_web_sm")

# --- Extract NER from full text ---
def extract_entities(text: str) -> List[Dict]:
    doc = nlp(text)
    return [{"entity": ent.text, "label": ent.label_} for ent in doc.ents]

# --- Generate a single embedding for the full text ---
def embed_full_text(text: str, dim: int = 3072) -> List[float]:
    try:
        response = genai.embed_content(
            model="models/embedding-001",
            content=text,
            task_type="SEMANTIC_SIMILARITY",
            output_dimensionality=dim
        )
        return response["embedding"]
    except Exception as e:
        print(f"Embedding error: {e}")
        return []

# --- Process the text and link each entity to full-text embedding ---
def process_text_with_full_context(text: str):
    print(f"\n📘 Input Text:\n{text.strip()[:500]}...")  # Print part of the text

    entities = extract_entities(text)
    if not entities:
        print("No named entities found.")
        return

    print("\n🔍 Extracted Entities:")
    for i, e in enumerate(entities):
        print(f"🔹 Entity {i+1}: {e['entity']} ({e['label']})")

    print("\n🧠 Generating full-text contextual embedding...")
    embedding = embed_full_text(text)

    if embedding:
        print(f"✅ Full-text embedding generated. Length: {len(embedding)}")

        # You can now link every entity to this full-context embedding
        print("\n🔗 Linking entities to document-level embedding...")
        for i, e in enumerate(entities):
            print(f"Entity {i+1}: {e['entity']} → uses full-text embedding (3072-dim)\n")
    else:
        print("❌ Failed to generate full-text embedding.")

# --- Sample Input ---
sample_text = """
Disney Land Health System
Savannah, Georgia

Operative Note
Signed

Patient: Mouse, Mini                 MR#: F29
DOB: 09/30/1952                     Acct: J045
Age/Sex: 70 / F                     ADM Date: 10/10/22
Loc: C-3C
Attending Dr: Donald Duck MD

Initialization Date/Time: 10/10/22, 1106

Pre-Op Diagnosis:
  1. Left Breast Invasive Ductal Cancer

Post-Op Diagnosis:
  Same

Procedure:
  1. Left Breast Scout Radar localized partial mastectomy with margins
  2. Local Tissue Rearrangement of left breast measuring 4 cm x 4 cm

Attending Surgeon: Donald Duck, MD
Anesthesia: GET and Local
EBL: Minimal

Specimens:
  1. Left breast cancer with long stitch lateral; short stitch superior;
     2 clips medial; 1 clip inferior

Disposition: To recovery room; stable

Brief History: Mini Mouse is a very pleasant 70 year old female with a biopsy
proven left breast cancer. The patient and I have had extensive discussions
regarding her diagnosis and treatment options. She has consented to breast
conservation and thoroughly understands the risks and benefits of this approach,
including risks of lymphedema and axillary numbness as well as the risk of a
second surgery for positive margins.
"""

# --- Run ---
process_text_with_full_context(sample_text)



📘 Input Text:
Disney Land Health System
Savannah, Georgia

Operative Note
Signed

Patient: Mouse, Mini                 MR#: F29
DOB: 09/30/1952                     Acct: J045
Age/Sex: 70 / F                     ADM Date: 10/10/22
Loc: C-3C
Attending Dr: Donald Duck MD

Initialization Date/Time: 10/10/22, 1106

Pre-Op Diagnosis:
  1. Left Breast Invasive Ductal Cancer

Post-Op Diagnosis:
  Same

Procedure:
  1. Left Breast Scout Radar localized partial mastectomy with margins
  2. Local Tissue Rearrangement of...

🔍 Extracted Entities:
🔹 Entity 1: Disney Land Health System (ORG)
🔹 Entity 2: Savannah (GPE)
🔹 Entity 3: Georgia (GPE)
🔹 Entity 4: Mouse (PERSON)
🔹 Entity 5: Mini (PERSON)
🔹 Entity 6: 70 / (CARDINAL)
🔹 Entity 7: 10/10/22 (CARDINAL)
🔹 Entity 8: Loc (PERSON)
🔹 Entity 9: Donald Duck MD (PERSON)
🔹 Entity 10: 10/10/22, 1106 (DATE)
🔹 Entity 11: 2 (CARDINAL)
🔹 Entity 12: 4 cm x 4 (QUANTITY)
🔹 Entity 13: Donald Duck (PERSON)
🔹 Entity 14: MD (GPE)
🔹 Entity 15: Anesthesia (GPE)
🔹 Entit

In [5]:
#Embeeding genetated by taking full document text approach
import spacy
import google.generativeai as genai
from typing import List, Dict

# === Configure the Gemini SDK ===
# In GCP (Enterprise or Colab), this will use the default service account
#genai.configure()

# === Load spaCy English model for NER ===
nlp = spacy.load("en_core_web_sm")

# === Extract Named Entities (avoid duplicates) ===
def extract_entities(text: str) -> List[Dict]:
    doc = nlp(text)
    seen = set()
    entities = []
    for ent in doc.ents:
        key = (ent.text, ent.label_)
        if key not in seen:
            seen.add(key)
            entities.append({
                "entity": ent.text,
                "label": ent.label_,
            })
    return entities

# === Embed each entity with full document context ===
def embed_entity_with_full_context(entity: str, full_text: str, dim: int = 3072) -> List[float]:
    prompt = f"Entity: {entity}\nContext: {full_text}"
    try:
        response = genai.embed_content(
            model="models/embedding-001",
            content=prompt,
            task_type="SEMANTIC_SIMILARITY",
            output_dimensionality=dim
        )
        return response["embedding"]
    except Exception as e:
        print(f"❌ Embedding failed for entity '{entity}': {e}")
        return []

# === Main processing function ===
def process_text_for_entity_embeddings(text: str):
    print(f"\n📘 Full Text Sample:\n{text[:300]}...\n")

    entities = extract_entities(text)
    if not entities:
        print("No named entities found.")
        return

    print("🔍 Found Entities and Contextual Embeddings:\n")
    for i, ent in enumerate(entities):
        print(f"🔹 Entity {i+1}: {ent['entity']} ({ent['label']})")
        embedding = embed_entity_with_full_context(ent["entity"], text)
        if embedding:
            print(f"   ✅ Embedding generated (length: {len(embedding)})")
            print(f"   🔸 Sample: {embedding[:5]}...\n")
        else:
            print(f"   ❌ Failed to generate embedding.\n")

# === Sample Input ===
sample_text = """
Disney Land Health System
Savannah, Georgia

Operative Note

Patient: Mouse, Mini                 MR#: F29
DOB: 09/30/1952                     Acct: J045
Age/Sex: 70 / F                     ADM Date: 10/10/22
Attending Dr: Donald Duck MD

Pre-Op Diagnosis: Left Breast Invasive Ductal Cancer

Procedure: Left Breast partial mastectomy with margins and tissue rearrangement.

Brief History: Mini Mouse is a very pleasant 70 year old female with a biopsy
proven left breast cancer. She has consented to breast conservation and understands the risks.
"""

# === Run the process ===
process_text_for_entity_embeddings(sample_text)



📘 Full Text Sample:

Disney Land Health System
Savannah, Georgia

Operative Note

Patient: Mouse, Mini                 MR#: F29
DOB: 09/30/1952                     Acct: J045
Age/Sex: 70 / F                     ADM Date: 10/10/22
Attending Dr: Donald Duck MD

Pre-Op Diagnosis: Left Breast Invasive Ductal Cancer

Proced...

🔍 Found Entities and Contextual Embeddings:

🔹 Entity 1: Disney Land Health System (ORG)
❌ Embedding failed for entity 'Disney Land Health System': 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.
   ❌ Failed to generate embedding.

🔹 Entity 2: Savannah (GPE)
❌ Embedding failed for entity 'Savannah': 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `gena